# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [4]:
import pandas as pd
import plotly as py
import cufflinks as cf
from lifelines import KaplanMeierFitter

cf.go_offline()

In [1]:
pip install cufflinks

  Using cached https://files.pythonhosted.org/packages/9a/53/f696e4480b1d1de3b1523991dea71cf417c8b19fe70c704da164f3f90972/colorlover-0.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ff/75/3982bac5076d0ce6d23103c03840fcaec90c533409f9d82c19f54512a38a/plotly-3.10.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lifelines

  Stored in directory: C:\Users\Nancy\AppData\Local\pip\Cache\wheels\42\62\66\1121afe23ff96af4e452e0d15e68761e3f605952ee075ca99f
Successfully built autograd
Note: you may need to restart the kernel to use updated packages.


In [16]:
data = pd.read_csv('../data/attrition.csv')
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [13]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

In [27]:
rates = survival(data, 'Gender',  'Age', 'Attrition' )

rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Age and Gender')

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [45]:
rates = survival(data, 'Gender',  'YearsAtCompany', 'Attrition' )

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by YearsAtCompany and Gender')

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [32]:
rates = survival(data, 'Gender',  'YearsSinceLastPromotion', 'Attrition' )

rates.iplot(kind='line', xTitle='YearsSinceLastPromotion', yTitle='Retention Rate',
            title='Retention Rates by employee s last promotion and Gender')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [46]:
rates = survival(data, 'MaritalStatus',  'YearsAtCompany', 'Attrition' )

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by different marital status by number of years at the company')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [36]:
rates = survival(data, 'MaritalStatus',  'Age', 'Attrition' )

rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by marital status and age')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [47]:
data['Marital+Gender'] = data['MaritalStatus'] + ' ' + data['Gender']

rates = survival(data, 'Marital+Gender', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Retention',
            title='Retention Rates by Age, Marital Status, and Gender')

## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [49]:
rates = survival(data, 'JobLevel','YearsAtCompany', 'Attrition' )

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Job Satisfaction')

## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [51]:
rates = survival(data, 'Department','YearsAtCompany', 'Attrition' )

rates.iplot(kind='line', xTitle='', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Department')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [52]:

sales = data[data['Department']=='Sales']

rates = survival(sales, 'JobRole', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure', yTitle='Retention',
            title='Retention Rates by Tenure and Job Role')

## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [53]:

pay_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
data['PayGrade'] = pd.qcut(data['HourlyRate'], 5, labels=pay_labels)

rates = survival(data, 'PayGrade', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Retention',
            title='Retention Rates by Age and Pay Grade')

## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.